# Libraries

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from functools import partial
from collections import Counter
import seaborn as sns

from joblib import load

# modeling
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import recall_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# ignore warnings
import warnings
warnings.filterwarnings(action="ignore")

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

# Get dataset

In [2]:
# get pickle file
providers = load('./data/Providers_Third.pkl')

In [3]:
providers

,PotentialFraud,Perc_Outpatient,DualPatientProvider,Perc_DualPatientType,Ratio_ClaimsPerPatient,Ratio_ClaimsPerAttPhys,Perc_ClaimsPerTopFraudState,Perc_HasTop5AdmtCode,Perc_MultHospAttPhys,Perc_MultHospOperPhys,Perc_MultHospOtherPhys,IP_Perc_Duplicates,IP_Count_UniquePatients,IP_Count_UniqueState,IP_Mean_AgeAtClaim,IP_Perc_HasDied,IP_Perc_GenderZero,IP_Perc_RaceOne,IP_Perc_RaceTwo,IP_Perc_RaceThree,IP_Mean_NumChronicConds,IP_Perc_Alzheimers_Chronic,IP_Perc_Cancer_Chronic,IP_Perc_Depression_Chronic,IP_Perc_Diabetes_Chronic,IP_Perc_HeartFailure_Chronic,IP_Perc_IschemicHeart_Chronic,IP_Perc_KidneyDisease_Chronic,IP_Perc_ObstrPulmonary_Chronic,IP_Perc_Osteoporosis_Chronic,IP_Perc_RheumatoidArthritis_Chronic,IP_Perc_Stroke_Chronic,IP_Perc_HasRenalDisease,IP_Mean_ClaimDuration,IP_Mean_AdmitDuration,IP_Mean_NoOfMonths_PartACov,IP_Mean_NoOfMonths_PartBCov,IP_Mean_ClaimCost,IP_Mean_DailyClaimCost,IP_Mean_DeductibleAmtPaid,IP_Mean_InscClaimAmtReimbursed,IP_Mean_InsReimbursementRatio,IP_Mean_AnnualDeductibleAmt,IP_Mean_AnnualReimbursementAmt,IP_Perc_No_ProcCode,IP_Perc_HasAllPhys,IP_Perc_HasNoPhys,IP_Perc_MultHosp,OP_Perc_Duplicates,OP_Count_UniquePatients,OP_Count_UniqueState,OP_Mean_AgeAtClaim,OP_Perc_HasDied,OP_Perc_GenderZero,OP_Perc_RaceOne,OP_Perc_RaceTwo,OP_Perc_RaceThree,OP_Mean_NumChronicConds,OP_Perc_Alzheimers_Chronic,OP_Perc_Cancer_Chronic,OP_Perc_Depression_Chronic,OP_Perc_Diabetes_Chronic,OP_Perc_HeartFailure_Chronic,OP_Perc_IschemicHeart_Chronic,OP_Perc_KidneyDisease_Chronic,OP_Perc_ObstrPulmonary_Chronic,OP_Perc_Osteoporosis_Chronic,OP_Perc_RheumatoidArthritis_Chronic,OP_Perc_Stroke_Chronic,OP_Perc_HasRenalDisease,OP_Mean_ClaimDuration,OP_Mean_NoOfMonths_PartACov,OP_Mean_NoOfMonths_PartBCov,OP_Mean_ClaimCost,OP_Mean_DailyClaimCost,OP_Mean_DeductibleAmtPaid,OP_Mean_InscClaimAmtReimbursed,OP_Mean_InsReimbursementRatio,OP_Mean_AnnualDeductibleAmt,OP_Mean_AnnualReimbursementAmt,OP_Perc_No_DiagCode,OP_Perc_HasAllPhys,OP_Perc_HasNoPhys,OP_Perc_MultHosp,IP_Mean_PatientsPerAttPhys,IP_Mean_PatientsPerOperPhys,IP_Mean_PatientsPerOtherPhys,IP_AgeRange,OP_Mean_PatientsPerAttPhys,OP_Mean_PatientsPerOperPhys,OP_Mean_PatientsPerOtherPhys,OP_AgeRange
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV51001,0,0.800000,1.0,0.400000,1.041667,1.785714,0.0,0.040000,0.240000,0.040000,0.000000,0.000000,5.0,1.0,77.600000,0.000000,0.400000,1.000000,0.000000,0.0,6.000000,0.400000,0.200000,0.800000,0.800000,0.800000,0.800000,0.800000,0.400000,0.000000,0.600000,0.400000,0.400000,6.000000,6.000000,12.000000,12.000000,20468.000000,4077.360000,1068.0,19400.000000,0.878279,897.120000,17606.000000,0.600000,0.2,0.0,0.160000,0.550000,19.0,1.0,77.950000,0.000000,0.350000,0.800000,0.200000,0.000000,5.450000,0.650000,0.200000,0.250000,0.850000,0.750000,0.950000,0.650000,0.400000,0.300000,0.250000,0.200000,0.300000,1.550000,12.000000,12.000000,382.000000,307.000000,0.000000,382.000000,1.000000,463.920000,2615.200000,0.000000,0.050000,0.000000,0.920000,1.25,1.0,1.0,26.0,2.000000,1.000000,1.285714,40.0
PRV51003,1,0.530303,1.0,0.500000,1.128205,3.000000,0.0,0.060606,0.007576,0.000000,0.000000,0.016129,53.0,3.0,69.935484,0.016129,0.338710,0.790323,0.209677,0.0,4.919355,0.516129,0.112903,0.403226,0.790323,0.580645,0.887097,0.629032,0.370968,0.209677,0.306452,0.112903,0.274194,6.161290,6.161290,11.806452,11.806452,10309.935484,2384.941628,1068.0,9241.935484,0.821059,931.424242,7568.181818,0.370968,0.0,0.0,0.045455,0.357143,66.0,3.0,68.371429,0.000000,0.471429,0.828571,0.157143,0.000000,4.214286,0.342857,0.042857,0.414286,0.728571,0.628571,0.814286,0.357143,0.257143,0.285714,0.271429,0.071429,0.171429,3.357143,11.828571,11.928571,467.714286,336.440760,1.000000,466.714286,0.994032,737.121212,2678.181818,0.000000,0.057143,0.000000,0.818182,29.50,20.0,0.0,55.0,1.590909,1.000000,1.136364,59.0
PRV51004,0,1.000000,0.0,0.208054,1.079710,3.921053,0.0,0.013423,0.167785,0.013423,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.

# Train test split

In [4]:
# we want to normalize and down/upsample after splitting.
scores = providers.drop(['PotentialFraud'], axis=1)
scores = pd.DataFrame(scores, columns = scores.columns)
decision = providers["PotentialFraud"]

Xtrain, Xtest, ytrain, ytest = train_test_split(scores, decision, 
                                                test_size = .30, random_state=0, 
                                                stratify = decision)


# get splitted train set to df for upsampling
providers_trainTestSplit_train = pd.concat([ytrain, Xtrain], axis=1)
display(providers_trainTestSplit_train)
display(providers_trainTestSplit_train['PotentialFraud'].value_counts())

,PotentialFraud,Perc_Outpatient,DualPatientProvider,Perc_DualPatientType,Ratio_ClaimsPerPatient,Ratio_ClaimsPerAttPhys,Perc_ClaimsPerTopFraudState,Perc_HasTop5AdmtCode,Perc_MultHospAttPhys,Perc_MultHospOperPhys,Perc_MultHospOtherPhys,IP_Perc_Duplicates,IP_Count_UniquePatients,IP_Count_UniqueState,IP_Mean_AgeAtClaim,IP_Perc_HasDied,IP_Perc_GenderZero,IP_Perc_RaceOne,IP_Perc_RaceTwo,IP_Perc_RaceThree,IP_Mean_NumChronicConds,IP_Perc_Alzheimers_Chronic,IP_Perc_Cancer_Chronic,IP_Perc_Depression_Chronic,IP_Perc_Diabetes_Chronic,IP_Perc_HeartFailure_Chronic,IP_Perc_IschemicHeart_Chronic,IP_Perc_KidneyDisease_Chronic,IP_Perc_ObstrPulmonary_Chronic,IP_Perc_Osteoporosis_Chronic,IP_Perc_RheumatoidArthritis_Chronic,IP_Perc_Stroke_Chronic,IP_Perc_HasRenalDisease,IP_Mean_ClaimDuration,IP_Mean_AdmitDuration,IP_Mean_NoOfMonths_PartACov,IP_Mean_NoOfMonths_PartBCov,IP_Mean_ClaimCost,IP_Mean_DailyClaimCost,IP_Mean_DeductibleAmtPaid,IP_Mean_InscClaimAmtReimbursed,IP_Mean_InsReimbursementRatio,IP_Mean_AnnualDeductibleAmt,IP_Mean_AnnualReimbursementAmt,IP_Perc_No_ProcCode,IP_Perc_HasAllPhys,IP_Perc_HasNoPhys,IP_Perc_MultHosp,OP_Perc_Duplicates,OP_Count_UniquePatients,OP_Count_UniqueState,OP_Mean_AgeAtClaim,OP_Perc_HasDied,OP_Perc_GenderZero,OP_Perc_RaceOne,OP_Perc_RaceTwo,OP_Perc_RaceThree,OP_Mean_NumChronicConds,OP_Perc_Alzheimers_Chronic,OP_Perc_Cancer_Chronic,OP_Perc_Depression_Chronic,OP_Perc_Diabetes_Chronic,OP_Perc_HeartFailure_Chronic,OP_Perc_IschemicHeart_Chronic,OP_Perc_KidneyDisease_Chronic,OP_Perc_ObstrPulmonary_Chronic,OP_Perc_Osteoporosis_Chronic,OP_Perc_RheumatoidArthritis_Chronic,OP_Perc_Stroke_Chronic,OP_Perc_HasRenalDisease,OP_Mean_ClaimDuration,OP_Mean_NoOfMonths_PartACov,OP_Mean_NoOfMonths_PartBCov,OP_Mean_ClaimCost,OP_Mean_DailyClaimCost,OP_Mean_DeductibleAmtPaid,OP_Mean_InscClaimAmtReimbursed,OP_Mean_InsReimbursementRatio,OP_Mean_AnnualDeductibleAmt,OP_Mean_AnnualReimbursementAmt,OP_Perc_No_DiagCode,OP_Perc_HasAllPhys,OP_Perc_HasNoPhys,OP_Perc_MultHosp,IP_Mean_PatientsPerAttPhys,IP_Mean_PatientsPerOperPhys,IP_Mean_PatientsPerOtherPhys,IP_AgeRange,OP_Mean_PatientsPerAttPhys,OP_Mean_PatientsPerOperPhys,OP_Mean_PatientsPerOtherPhys,OP_AgeRange
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV52041,1,1.000000,0.0,0.245623,2.053377,235.625000,0.037666,0.021751,0.115119,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,389.901326,3762.005305,0.000000,0.000000,0.0,0.041910,0.430960,918.0,17.0,74.118302,0.004775,0.410610,0.666667,0.000000,0.142857,4.338462,0.363395,0.138462,0.406897,0.663660,0.592573,0.766578,0.372414,0.328912,0.327321,0.283820,0.094430,0.198939,2.445093,11.980371,11.970822,323.856764,242.538986,2.912467,320.944297,0.984813,653.856764,2449.395225,0.016446,0.093369,0.003714,0.846154,0.000000,0.00,0.000000,0.0,203.125000,51.000000,288.000000,74.0
PRV51771,0,1.000000,0.0,0.425000,1.176471,40.000000,1.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,507.300000,3096.000000,0.000000,0.000000,0.0,0.050000,0.525000,34.0,1.0,73.725000,0.000000,0.325000,1.000000,0.000000,0.000000,3.525000,0.325000,0.100000,0.375000,0.575000,0.475000,0.625000,0.325000,0.325000,0.200000,0.150000,0.050000,0.175000,1.675000,11.700000,12.000000,214.750000,143.321429,1.500000,213.250000,0.974969,632.500000,1709.250000,0.000000,0.100000,0.000000,0.925000,0.000000,0.00,0.000000,0.0,34.000000,6.000000,15.000000,72.0
PRV52079,0,0.787879,1.0,0.272727,1.064516,1.500000,0.000000,0.030303,0.090909,0.030303,0.000000,0.142857,7.0,2.0,68.714286,0.

0    3433
1     354
Name: PotentialFraud, dtype: int64

# Upsampling

In [5]:
### UPSAMPLING
# Separate majority and minority classes
df_majority = providers_trainTestSplit_train[providers_trainTestSplit_train.PotentialFraud==0]
df_minority = providers_trainTestSplit_train[providers_trainTestSplit_train.PotentialFraud==1]

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                # to match majority class
                                 n_samples=providers_trainTestSplit_train['PotentialFraud'].value_counts()[0],
                                 random_state=0) # reproducible results

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

# Display new class counts
display(df_upsampled.PotentialFraud.value_counts())

# separate into Xtrain and Ytrain
Xtrain_upsampled = df_upsampled.drop(['PotentialFraud'], axis=1)
ytrain_upsampled = df_upsampled['PotentialFraud']

1    3433
0    3433
Name: PotentialFraud, dtype: int64

# Modeling

In [6]:
# Stratified Cross Validation needs to be applied 
skf = StratifiedKFold(n_splits = 3, random_state = 0, shuffle = True)

### 1. Logistic regression

In [7]:
logreg = LogisticRegression(random_state=0, class_weight='balanced', penalty='l1', solver='liblinear')

param_grid = {'C': np.logspace(-3,1,100)}

logregCV = GridSearchCV(logreg, 
                        param_grid = param_grid, 
                        scoring='recall',
                        cv=skf).fit(Xtrain,ytrain)

print("Best parameters: {}".format(logregCV.best_params_))
print("Best cross-validation score: {:.2f}".format(logregCV.best_score_))
print("Best estimator:\n{}".format(logregCV.best_estimator_))

print('--'*30)
print("Train recall score: {}".format(recall_score(ytrain, logregCV.best_estimator_.predict(Xtrain))))
print("Train recall score: {}".format(recall_score(ytest, logregCV.best_estimator_.predict(Xtest))))

print('--'*30)
print('Confusion matrix')
print(confusion_matrix(ytest,logregCV.predict(Xtest)))

Best parameters: {'C': 0.22051307399030456}
Best cross-validation score: 0.87
Best estimator:
LogisticRegression(C=0.22051307399030456, class_weight='balanced', penalty='l1',
                   random_state=0, solver='liblinear')
------------------------------------------------------------
Train recall score: 0.9180790960451978
Train recall score: 0.8881578947368421
------------------------------------------------------------
Confusion matrix
[[1278  193]
 [  17  135]]


### 2. Random forest

In [29]:
rf_param_grid = {
    'max_depth': range(1, 10), 
    'max_features': ['auto', 'log2'], 
    'min_samples_leaf': [4,5,6], 
    'min_samples_split': [1,2,3,4,5], 
    'n_estimators': [300,350,375,400]
}

rf_grid_search = RandomizedSearchCV(RandomForestClassifier(random_state=0, 
                                                  class_weight='balanced'), 
                                    param_distributions=rf_param_grid, cv=skf, scoring='recall', 
                                    random_state=0, return_train_score=True)
rf_grid_search.fit(Xtrain, ytrain)


print("Best parameters: {}".format(rf_grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(rf_grid_search.best_score_))
print("Best estimator:\n{}".format(rf_grid_search.best_estimator_))

print('--'*30)
print("Train recall score: {}".format(recall_score(ytrain, rf_grid_search.best_estimator_.predict(Xtrain))))
print("Train recall score: {}".format(recall_score(ytest, rf_grid_search.best_estimator_.predict(Xtest))))

print('--'*30)
print('Confusion matrix')
print(confusion_matrix(ytest,rf_grid_search.predict(Xtest)))

Best parameters: {'n_estimators': 350, 'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 'auto', 'max_depth': 3}
Best cross-validation score: 0.86
Best estimator:
RandomForestClassifier(class_weight='balanced', max_depth=3, min_samples_leaf=5,
                       min_samples_split=5, n_estimators=350, random_state=0)
------------------------------------------------------------
Train recall score: 0.9096045197740112
Train recall score: 0.9013157894736842
------------------------------------------------------------
Confusion matrix
[[1235  236]
 [  15  137]]


### 3. Support vector machine

In [9]:
svm_param_grid = {'C': np.logspace(-3,1,100)}

svm_rand_search = RandomizedSearchCV(SVC(random_state=0, class_weight='balanced'),
                                     svm_param_grid, 
                                     cv=skf, 
                                     scoring='recall')
svm_rand_search.fit(Xtrain, ytrain)

print("Best parameters: {}".format(svm_rand_search.best_params_))
print("Best cross-validation score: {:.2f}".format(svm_rand_search.best_score_))
print("Best estimator:\n{}".format(svm_rand_search.best_estimator_))

print('--'*30)
print("Train recall score: {}".format(recall_score(ytrain, svm_rand_search.best_estimator_.predict(Xtrain))))
print("Train recall score: {}".format(recall_score(ytest, svm_rand_search.best_estimator_.predict(Xtest))))

print('--'*30)
print('Confusion matrix')
print(confusion_matrix(ytest,svm_rand_search.predict(Xtest)))

Best parameters: {'C': 0.004862601580065354}
Best cross-validation score: 0.87
Best estimator:
SVC(C=0.004862601580065354, class_weight='balanced', random_state=0)
------------------------------------------------------------
Train recall score: 0.867231638418079
Train recall score: 0.8552631578947368
------------------------------------------------------------
Confusion matrix
[[1046  425]
 [  22  130]]


### 4. Gaussian Naive Bayes

In [10]:
gnb_param_grid = {'var_smoothing': np.logspace(-30,1,100)}

gnb_rand_search = RandomizedSearchCV(GaussianNB(),
                                     gnb_param_grid, 
                                     cv=skf, 
                                     scoring='recall')
gnb_rand_search.fit(Xtrain, ytrain)

print("Best parameters: {}".format(gnb_rand_search.best_params_))
print("Best cross-validation score: {:.2f}".format(gnb_rand_search.best_score_))
print("Best estimator:\n{}".format(gnb_rand_search.best_estimator_))

print('--'*30)
print("Train recall score: {}".format(recall_score(ytrain, gnb_rand_search.best_estimator_.predict(Xtrain))))
print("Train recall score: {}".format(recall_score(ytest, gnb_rand_search.best_estimator_.predict(Xtest))))

print('--'*30)
print('Confusion matrix')
print(confusion_matrix(ytest,gnb_rand_search.predict(Xtest)))

Best parameters: {'var_smoothing': 2.8480358684357933e-22}
Best cross-validation score: 0.82
Best estimator:
GaussianNB(var_smoothing=2.8480358684357933e-22)
------------------------------------------------------------
Train recall score: 0.8192090395480226
Train recall score: 0.8092105263157895
------------------------------------------------------------
Confusion matrix
[[1233  238]
 [  29  123]]


In [11]:
gnb_param_grid = {'var_smoothing': np.logspace(-10,1,100)}

gnb_rand_search = RandomizedSearchCV(GaussianNB(),
                                     gnb_param_grid, 
                                     cv=skf, 
                                     scoring='recall')
gnb_rand_search.fit(Xtrain_upsampled, ytrain_upsampled)

print("Best parameters: {}".format(gnb_rand_search.best_params_))
print("Best cross-validation score: {:.2f}".format(gnb_rand_search.best_score_))
print("Best estimator:\n{}".format(gnb_rand_search.best_estimator_))

print('--'*30)
print("Train recall score: {}".format(recall_score(ytrain_upsampled, 
                                                   gnb_rand_search.best_estimator_.predict(Xtrain_upsampled))))
print("Train recall score: {}".format(recall_score(ytest, gnb_rand_search.best_estimator_.predict(Xtest))))

print('--'*30)
print('Confusion matrix')
print(confusion_matrix(ytest,gnb_rand_search.predict(Xtest)))

Best parameters: {'var_smoothing': 2.1544346900318866e-09}
Best cross-validation score: 0.80
Best estimator:
GaussianNB(var_smoothing=2.1544346900318866e-09)
------------------------------------------------------------
Train recall score: 0.8054180017477425
Train recall score: 0.7828947368421053
------------------------------------------------------------
Confusion matrix
[[1326  145]
 [  33  119]]


### 5. KNN

In [12]:
knn_param_grid = {'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                  'leaf_size': [10,30,50],
                  'n_neighbors': [3,5,7],
                  'weights': ['uniform', 'distance']
                 }

knn_rand_search = RandomizedSearchCV(KNeighborsClassifier(),
                                     knn_param_grid, 
                                     cv=skf, 
                                     scoring='recall')
knn_rand_search.fit(Xtrain, ytrain)

print("Best parameters: {}".format(knn_rand_search.best_params_))
print("Best cross-validation score: {:.2f}".format(knn_rand_search.best_score_))
print("Best estimator:\n{}".format(knn_rand_search.best_estimator_))

print('--'*30)
print("Train recall score: {}".format(recall_score(ytrain, knn_rand_search.best_estimator_.predict(Xtrain))))
print("Train recall score: {}".format(recall_score(ytest, knn_rand_search.best_estimator_.predict(Xtest))))

print('--'*30)
print('Confusion matrix')
print(confusion_matrix(ytest,knn_rand_search.predict(Xtest)))

Best parameters: {'weights': 'uniform', 'n_neighbors': 3, 'leaf_size': 50, 'algorithm': 'brute'}
Best cross-validation score: 0.29
Best estimator:
KNeighborsClassifier(algorithm='brute', leaf_size=50, n_neighbors=3)
------------------------------------------------------------
Train recall score: 0.5988700564971752
Train recall score: 0.4342105263157895
------------------------------------------------------------
Confusion matrix
[[1410   61]
 [  86   66]]


In [13]:
knn_param_grid = {'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                  'leaf_size': [10,30,50],
                  'n_neighbors': [3,5,7],
                  'weights': ['uniform', 'distance']
                 }

knn_rand_search = RandomizedSearchCV(KNeighborsClassifier(),
                                     knn_param_grid, 
                                     cv=skf, 
                                     scoring='recall')
knn_rand_search.fit(Xtrain_upsampled, ytrain_upsampled)

print("Best parameters: {}".format(knn_rand_search.best_params_))
print("Best cross-validation score: {:.2f}".format(knn_rand_search.best_score_))
print("Best estimator:\n{}".format(knn_rand_search.best_estimator_))

print('--'*30)
print("Train recall score: {}".format(recall_score(ytrain_upsampled,
                                                   knn_rand_search.best_estimator_.predict(Xtrain_upsampled))))
print("Train recall score: {}".format(recall_score(ytest, knn_rand_search.best_estimator_.predict(Xtest))))

print('--'*30)
print('Confusion matrix')
print(confusion_matrix(ytest,knn_rand_search.predict(Xtest)))

Best parameters: {'weights': 'distance', 'n_neighbors': 3, 'leaf_size': 30, 'algorithm': 'auto'}
Best cross-validation score: 1.00
Best estimator:
KNeighborsClassifier(n_neighbors=3, weights='distance')
------------------------------------------------------------
Train recall score: 1.0
Train recall score: 0.6052631578947368
------------------------------------------------------------
Confusion matrix
[[1317  154]
 [  60   92]]


### 6. Gradient boosting

In [15]:
gb = GradientBoostingClassifier(random_state = 0, max_depth = 2)
gb_param_grid = {'n_estimators': [500,1000,2000,2500,3000],
                'max_features': [10,13,15,20],
                'learning_rate': [0.005, 0.01, 0.015, 0.02],
                    'min_samples_split': [17,20,25],
                    'min_samples_leaf' : [17,20,25]}

gb_rand_search = RandomizedSearchCV(gb, 
                                    gb_param_grid, 
                                    cv= skf, 
                                    n_jobs = 5, 
                                    verbose = 1,
                                    return_train_score = True,
                                    scoring = 'recall')

gb_rand_search.fit(Xtrain, ytrain)

print("Best parameters: {}".format(gb_rand_search.best_params_))
print("Best cross-validation score: {:.2f}".format(gb_rand_search.best_score_))
print("Best estimator:\n{}".format(gb_rand_search.best_estimator_))

print('--'*30)
print("Train recall score: {}".format(recall_score(ytrain, gb_rand_search.best_estimator_.predict(Xtrain))))
print("Train recall score: {}".format(recall_score(ytest, gb_rand_search.best_estimator_.predict(Xtest))))

print('--'*30)
print('Confusion matrix')
print(confusion_matrix(ytest,gb_rand_search.predict(Xtest)))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  30 out of  30 | elapsed:   39.9s finished


Best parameters: {'n_estimators': 3000, 'min_samples_split': 25, 'min_samples_leaf': 25, 'max_features': 20, 'learning_rate': 0.01}
Best cross-validation score: 0.53
Best estimator:
GradientBoostingClassifier(learning_rate=0.01, max_depth=2, max_features=20,
                           min_samples_leaf=25, min_samples_split=25,
                           n_estimators=3000, random_state=0)
------------------------------------------------------------
Train recall score: 0.7655367231638418
Train recall score: 0.5131578947368421
------------------------------------------------------------
Confusion matrix
[[1445   26]
 [  74   78]]


In [19]:
gb = GradientBoostingClassifier(random_state = 0, max_depth = 2)
gb_param_grid = {'n_estimators': [3000,3100,3200,3250],
                'max_features': [19,20,21],
                'learning_rate': [0.03,0.035,0.04],
                    'min_samples_split': [25,26,27,28],
                    'min_samples_leaf' : [14,15,16]}

gb_rand_search = RandomizedSearchCV(gb, 
                                    gb_param_grid, 
                                    cv= skf, 
                                    n_jobs = 5, 
                                    verbose = 1,
                                    return_train_score = True,
                                    scoring = 'recall')

gb_rand_search.fit(Xtrain_upsampled, ytrain_upsampled)

print("Best parameters: {}".format(gb_rand_search.best_params_))
print("Best cross-validation score: {:.2f}".format(gb_rand_search.best_score_))
print("Best estimator:\n{}".format(gb_rand_search.best_estimator_))

print('--'*30)
print("Train recall score: {}".format(recall_score(ytrain_upsampled,
                                                   gb_rand_search.best_estimator_.predict(Xtrain_upsampled))))
print("Train recall score: {}".format(recall_score(ytest, gb_rand_search.best_estimator_.predict(Xtest))))

print('--'*30)
print('Confusion matrix')
print(confusion_matrix(ytest,gb_rand_search.predict(Xtest)))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  30 out of  30 | elapsed:  2.5min finished


Best parameters: {'n_estimators': 3200, 'min_samples_split': 27, 'min_samples_leaf': 15, 'max_features': 20, 'learning_rate': 0.04}
Best cross-validation score: 1.00
Best estimator:
GradientBoostingClassifier(learning_rate=0.04, max_depth=2, max_features=20,
                           min_samples_leaf=15, min_samples_split=27,
                           n_estimators=3200, random_state=0)
------------------------------------------------------------
Train recall score: 1.0
Train recall score: 0.6710526315789473
------------------------------------------------------------
Confusion matrix
[[1420   51]
 [  50  102]]
